Deep Learning
=============

Assignment 6
------------

After training a skip-gram model in `5_word2vec.ipynb`, the goal of this notebook is to train a LSTM character model over [Text8](http://mattmahoney.net/dc/textdata) data.

In [318]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

In [319]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified %s' % filename)
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [320]:
def read_data(filename):
  f = zipfile.ZipFile(filename)
  for name in f.namelist():
    return tf.compat.as_str(f.read(name))
  f.close()
  
text = read_data(filename)
print('Data size %d' % len(text))

Data size 100000000


Create a small validation set.

In [321]:
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

99999000 ons anarchists advocate social relations based upon voluntary as
1000  anarchism originated as a term of abuse first used against earl


Utility functions to map characters to vocabulary IDs and back.

In [322]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0])

def char2id(char):
  if char in string.ascii_lowercase:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  else:
    print('Unexpected character: %s' % char)
    return 
  
def id2char(dictid):
  if dictid > 0:
    return chr(dictid + first_letter - 1)
  else:
    return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(26), id2char(0))

Unexpected character: ï
1 26 0 None
a z  


Function to generate a training batch for the LSTM model.

In [170]:
batch_size=64
num_unrollings=10

class BatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // batch_size
    self._cursor = [ offset * segment for offset in range(batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
    for b in range(self._batch_size):
      batch[b, char2id(self._text[self._cursor[b]])] = 1.0
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

def characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, characters(b))]
  return s

train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1)

print(batches2string(train_batches.next()))
print(batches2string(train_batches.next()))
print(batches2string(valid_batches.next()))
print(batches2string(valid_batches.next()))

['ons anarchi', 'when milita', 'lleria arch', ' abbeys and', 'married urr', 'hel and ric', 'y and litur', 'ay opened f', 'tion from t', 'migration t', 'new york ot', 'he boeing s', 'e listed wi', 'eber has pr', 'o be made t', 'yer who rec', 'ore signifi', 'a fierce cr', ' two six ei', 'aristotle s', 'ity can be ', ' and intrac', 'tion of the', 'dy to pass ', 'f certain d', 'at it will ', 'e convince ', 'ent told hi', 'ampaign and', 'rver side s', 'ious texts ', 'o capitaliz', 'a duplicate', 'gh ann es d', 'ine january', 'ross zero t', 'cal theorie', 'ast instanc', ' dimensiona', 'most holy m', 't s support', 'u is still ', 'e oscillati', 'o eight sub', 'of italy la', 's the tower', 'klahoma pre', 'erprise lin', 'ws becomes ', 'et in a naz', 'the fabian ', 'etchy to re', ' sharman ne', 'ised empero', 'ting in pol', 'd neo latin', 'th risky ri', 'encyclopedi', 'fense the a', 'duating fro', 'treet grid ', 'ations more', 'appeal of d', 'si have mad']
['ists advoca', 'ary governm', 'hes nat

In [6]:
def logprob(predictions, labels):
  """Log-probability of the true labels in a predicted batch."""
  predictions[predictions < 1e-10] = 1e-10
  return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
  """Sample one element from a distribution assumed to be an array of normalized
  probabilities.
  """
  r = random.uniform(0, 1)
  s = 0
  for i in range(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def sample(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
  return b/np.sum(b, 1)[:,None]

Simple LSTM Model.

---
Problem 1
---------

You might have noticed that the definition of the LSTM cell involves 4 matrix multiplications with the input, and 4 matrix multiplications with the output. Simplify the expression by using a single matrix multiply for each, and variables that are 4 times larger.

---

---
Problem 2
---------

We want to train a LSTM over bigrams, that is pairs of consecutive characters like 'ab' instead of single characters like 'a'. Since the number of possible bigrams is large, feeding them directly to the LSTM using 1-hot encodings will lead to a very sparse representation that is very wasteful computationally.

a- Introduce an embedding lookup on the inputs, and feed the embeddings to the LSTM cell instead of the inputs themselves.

b- Write a bigram-based LSTM, modeled on the character LSTM above.

c- Introduce Dropout. For best practices on how to use Dropout in LSTMs, refer to this [article](http://arxiv.org/abs/1409.2329).

---

See https://github.com/vanhuyz/udacity-dl/blob/master/6_lstm-Problem2.ipynb

---
Problem 3
---------

(difficult!)

Write a sequence-to-sequence LSTM which mirrors all the words in a sentence. For example, if your input is:

    the quick brown fox
    
the model should attempt to output:

    eht kciuq nworb xof
    
Refer to the lecture on how to put together a sequence-to-sequence model, as well as [this article](http://arxiv.org/abs/1409.3215) for best practices.

---

In [212]:
ord(string.ascii_lowercase[0])

97

In [323]:
vocabulary_size = len(string.ascii_lowercase) + 3 # [a-z] + ' ' + <go> + <eos>
first_letter = ord(string.ascii_lowercase[0])
GO_ID = 1
EOS_ID = 2

def char2id(char):
  if char in string.ascii_lowercase:
    return ord(char) - first_letter + 3
  elif char == ' ':
    return 0
  elif char == '<go>':
    return 1
  elif char == '<eos>':
    return 2
  else:
    print('Unexpected character: %s' % char)
    return 
  
def id2char(dictid):
  if dictid > 2:
    return chr(dictid + first_letter - 3)
  elif dictid == GO_ID:
    return '<go>'
  elif dictid == EOS_ID:
    return '<eos>'
  else:
    return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'), char2id('<go>'), char2id('<eos>'))
print(id2char(1), id2char(26), id2char(0))

def str2id(str):
  return [char2id(c) for c in str]

print(str2id('the quick brown fox'))
print(str2id('eht kciuq nworb xof'))

Unexpected character: ï
3 28 0 None 1 2
<go> x  
[22, 10, 7, 0, 19, 23, 11, 5, 13, 0, 4, 20, 17, 25, 16, 0, 8, 17, 26]
[7, 10, 22, 0, 13, 5, 11, 23, 19, 0, 16, 25, 17, 20, 4, 0, 26, 17, 8]


In [324]:
def reverse_word(word):
  return word[::-1]

def mirror_sequence(seq):
  return ' '.join(list(map(lambda word: reverse_word(word), seq.split(' '))))

print(mirror_sequence('ons anarchi'))

sno ihcrana


In [451]:
batch_size=64
sequence_length = 20

class BatchGenerator(object):
  def __init__(self, text, batch_size, sequence_length):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._sequence_length = sequence_length
    segment = self._text_size // batch_size
    self._cursor = [ offset * segment for offset in range(batch_size)]

  def next(self):
    """Generate the next array of batches from the data.
    """
    batches = [None] * self._batch_size
    for i in range(self._batch_size):
      batches[i] = self._text[self._cursor[i]:self._cursor[i]+self._sequence_length].ljust(self._sequence_length)
      self._cursor[i] = (self._cursor[i] + self._sequence_length - 1) % self._text_size
    return batches

def characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return [id2char(c) for c in np.argmax(probabilities, 1)]

def mirror_batches(batches):
   return [mirror_sequence(seq) for seq in batches]

train_batches = BatchGenerator(train_text, batch_size, sequence_length)
valid_batches = BatchGenerator(valid_text, 1, sequence_length)

batches = train_batches.next()
print(batches)
print(mirror_batches(batches))
print(valid_batches.next())
print(valid_batches.next())
print(valid_batches.next())


['ons anarchists advoc', 'when military govern', 'lleria arches nation', ' abbeys and monaster', 'married urraca princ', 'hel and richard baer', 'y and liturgical lan', 'ay opened for passen', 'tion from the nation', 'migration took place', 'new york other well ', 'he boeing seven six ', 'e listed with a glos', 'eber has probably be', 'o be made to recogni', 'yer who received the', 'ore significant than', 'a fierce critic of t', ' two six eight in si', 'aristotle s uncaused', 'ity can be lost as i', ' and intracellular i', 'tion of the size of ', 'dy to pass him a sti', 'f certain drugs conf', 'at it will take to c', 'e convince the pries', 'ent told him to name', 'ampaign and barred a', 'rver side standard f', 'ious texts such as e', 'o capitalize on the ', 'a duplicate of the o', 'gh ann es d hiver on', 'ine january eight ma', 'ross zero the lead c', 'cal theories classic', 'ast instance the non', ' dimensional analysi', 'most holy mormons be', 't s support or at le', 'u is still dis

In [454]:
lstm_size = 64

graph = tf.Graph()
with graph.as_default():
  encoder_inputs = list()
  decoder_inputs = list()
  train_labels = list()  

  for _ in range(sequence_length):
    encoder_inputs.append(tf.placeholder(tf.int32, shape=(None,)))
  for _ in range(sequence_length+1):
    decoder_inputs.append(tf.placeholder(tf.int32, shape=(None,)))
    train_labels.append(tf.placeholder(tf.int32, shape=(None,)))

  weights = [tf.ones_like(label, dtype=tf.float32) for label in train_labels]

  print('length of weight:', len(weights))
  print('shape of weight[0]:', weights[0].get_shape())

  # Use LSTM cell
  cell = tf.nn.rnn_cell.BasicLSTMCell(lstm_size)
  #outputs, states = tf.nn.seq2seq.basic_rnn_seq2seq(encoder_inputs, decoder_inputs, cell)
  with tf.variable_scope("train_test"):
    outputs, states = tf.nn.seq2seq.embedding_rnn_seq2seq(encoder_inputs,
                                                          decoder_inputs,
                                                          cell,
                                                          vocabulary_size, # num_encoder_symbols
                                                          vocabulary_size, # num_decoder_symbols
                                                          vocabulary_size, # embedding_size
                                                         )

  loss = tf.nn.seq2seq.sequence_loss(outputs, train_labels, weights) 
  optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

  # Predictions.
  train_predictions = tf.pack([tf.nn.softmax(output) for output in outputs])

  # Validation eval
  valid_encoder_inputs = list()
  valid_decoder_inputs = list()
  valid_labels = list()  

  for _ in range(sequence_length):
    valid_encoder_inputs.append(tf.placeholder(tf.int32, shape=(1,)))
  for _ in range(sequence_length+1):
    valid_decoder_inputs.append(tf.placeholder(tf.int32, shape=(1,)))
    valid_labels.append(tf.placeholder(tf.int32, shape=(1,)))
  valid_weights = [tf.ones_like(label, dtype=tf.float32) for label in valid_labels]
  with tf.variable_scope("train_test", reuse = True):
    valid_outputs, valid_states = tf.nn.seq2seq.embedding_rnn_seq2seq(valid_encoder_inputs,
                                                                     valid_decoder_inputs,
                                                                     cell,
                                                                     vocabulary_size, # num_encoder_symbols
                                                                     vocabulary_size, # num_decoder_symbols
                                                                     vocabulary_size, # embedding_size
                                                                     feed_previous=True
                                                                     )
  valid_predictions = tf.pack([output for output in valid_outputs])
  valid_loss = tf.nn.seq2seq.sequence_loss(valid_outputs, valid_labels, valid_weights) 

  print(valid_encoder_inputs[0].get_shape())

length of weight: 21
shape of weight[0]: (?,)
(1,)


In [455]:
def sample_prediction(predictions):
  return ''.join([characters(onehot)[0] for onehot in predictions])

with tf.Session(graph=graph) as sess:
  sess.run(tf.initialize_all_variables())

  for step in range(50001):
    feed_dict = dict()
    
    input_batches = train_batches.next()
    output_batches = mirror_batches(input_batches)
       
    feed_dict[decoder_inputs[0]] = [GO_ID] * batch_size
    for i in range(sequence_length):
      feed_dict[encoder_inputs[i]] = [char2id(seq[i]) for seq in input_batches]
      feed_dict[decoder_inputs[i+1]] = [char2id(seq[i]) for seq in output_batches]
      feed_dict[train_labels[i]] = [char2id(seq[i]) for seq in output_batches]
    feed_dict[train_labels[sequence_length]] = [EOS_ID] * batch_size
    
    _, l, predictions = sess.run([optimizer, loss, train_predictions], feed_dict=feed_dict)

    if step % 1000 == 0:
      print('Step %d:' % step)
      print('Training set:')
      print('  Perplexity       : ', l)
      print('  Input            : ', input_batches[0])
      print('  Correct output   : ', output_batches[0])
      print('  Generated output : ', first_prediction(predictions))
          
      valid_feed_dict = dict()
      valid_input_batches = valid_batches.next()
      valid_output_batches = mirror_batches(valid_input_batches)

      valid_feed_dict[valid_decoder_inputs[0]] = [GO_ID]
      for i in range(sequence_length):
        valid_feed_dict[valid_encoder_inputs[i]] = [char2id(valid_input_batches[0][i])]
        valid_feed_dict[valid_labels[i]] = [char2id(valid_output_batches[0][i])]
      valid_feed_dict[valid_labels[sequence_length]] = [EOS_ID]

      valid_l, valid_p = sess.run([valid_loss, valid_predictions], feed_dict=valid_feed_dict)
     
        
      print('Valid set:')
      print('  Perplexity       : ', valid_l)
      print('  Input            : ', valid_input_batches[0])
      print('  Correct output   : ', valid_output_batches[0])
      print('  Generated output : ', sample_prediction(valid_p))
      print("="*100)


Step 0:
Training set:
  Perplexity       :  3.37315
  Input            :  cate social relation
  Correct output   :  etac laicos noitaler
  Generated output :  ghh<eos> ddddgrrluuuwuuup
Valid set:
  Perplexity       :  3.3565
  Input            :  st early working cla
  Correct output   :  ts ylrae gnikrow alc
  Generated output :  kxxxvnpnpwkyyyyewpkkx
Step 1000:
Training set:
  Perplexity       :  2.66839
  Input            :  ictory but in one ni
  Correct output   :  yrotci tub ni eno in
  Generated output :         e   ei eei e <eos>
Valid set:
  Perplexity       :  2.88864
  Input            :  ass radicals includi
  Correct output   :  ssa slacidar idulcni
  Generated output :   ee e ee ee ee et <eos><eos><eos>
Step 2000:
Training set:
  Perplexity       :  2.39611
  Input            :   institutions that d
  Correct output   :   snoitutitsni taht d
  Generated output :   nii          ee   <eos><eos>
Valid set:
  Perplexity       :  3.01884
  Input            :  ing the diggers 